<a href="https://colab.research.google.com/github/Jasmine216/VMAF-Optimization/blob/master/Attention_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import tarfile
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9648825334460408353, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3884799868730972366
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11379259582280096335
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15701463552
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3458238356041388606
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [ ]:
pip list |grep tensor

tensorboard              2.2.2          
tensorboard-plugin-wit   1.6.0.post3    
tensorboardcolab         0.0.22         
tensorflow               2.2.0          
tensorflow-addons        0.8.3          
tensorflow-datasets      2.1.0          
tensorflow-estimator     2.2.0          
tensorflow-gcs-config    2.2.0          
tensorflow-hub           0.8.0          
tensorflow-metadata      0.22.2         
tensorflow-privacy       0.2.2          
tensorflow-probability   0.10.0         


# requirement

In [1]:
!pip uninstall tensorflow==1.13.1
!pip install tensorflow-gpu==1.13.1

Uninstalling tensorflow-2.3.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 345.2MB 45kB/s 
     |████████████████████████████████| 368kB 40.2MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 3.2MB 41.2MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0


In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import time
import h5py
#import OMCNN_2CLSTM as Network  # define the CNN
import random
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# BasicNet

In [4]:
class BasicNet(object):
  weight_decay = 5*1e-6
  weight_init = 0.1 #weight init for biasis
  leaky_alpha = 0.1
  is_training = False
  def __init__(self):
    self.pretrain_var_collection = []
    self.initial_var_collection = []
    self.trainable_var_collection = []
    self.var_rename = {}
    # self.weight_decay = FLAGS.weight_decay
    # self.weight_init = FLAGS.weight_init
    # self.leaky_alpha = FLAGS.leaky_alpha

  def leaky_relu(self, x, alpha, dtype=tf.float32):
    """leaky relu
    if x > 0:
      return x
    else:
      return alpha * x
    Args:
      x : Tensor
      alpha: float
    Return:
      y : Tensor
    """
    x = tf.cast(x, dtype=dtype)
    bool_mask = (x > 0)
    mask = tf.cast(bool_mask, dtype=dtype)
    return 1.0 * mask * x + alpha * (1 - mask) * x

  def get_bilinear(self, f_shape):
        width = f_shape[1]
        heigh = f_shape[0]
        f = width//2 + 1
        c = (2 * f - 1 - f % 2) / (2.0 * f)
        bilinear = np.zeros([f_shape[0], f_shape[1]])
        for x in range(width):
            for y in range(heigh):
                value = (1 - abs(x / f - c)) * (1 - abs(y / f - c))
                bilinear[x, y] = value
        weights = np.zeros(f_shape)
        bilinear = bilinear / (np.sum(bilinear)*f_shape[2])
        for i in range(f_shape[2]):
          for j in range(f_shape[3]):
            weights[:, :, i, j] = bilinear


        return weights

  def get_centermask(self,f_shape): # shape[batchsize, height, width, channals]
    width = f_shape[2]
    heigh = f_shape[1]
    midw = width//2
    midh = heigh//2
    distmatrix = np.zeros([heigh, width])
    for x in range(width):
      for y in range(heigh):
        value = np.sqrt((x - midw)**2+(y - midh)**2)
        distmatrix[x, y] = value
    distmatrix = distmatrix / np.max(distmatrix)
    distmatrix = 1 -  distmatrix
    distmatrix = distmatrix[np.newaxis,...,np.newaxis]
    # distmatrix = tf.expand_dims(distmatrix, 0)
    # distmatrix = tf.expand_dims(distmatrix, 3)
    # for a in range(f_shape[0]):
    #   for b in range(f_shape[3]):
    #     mask[a, :, :, b] = distmatrix
    return distmatrix


  def _activation_summary(self,x, name = None):
    """Helper to create summaries for activations.

    Creates a summary that provides a histogram of activations.
    Creates a summary that measure the sparsity of activations.

    Args:
      x: Tensor
    Returns:
      nothing
    """
    if name is None:
      tensor_name = x.op.name
    else:
      tensor_name = name
    tf.summary.histogram(tensor_name + '/activations', x)
    tf.summary.scalar(tensor_name + '/sparsity', tf.nn.zero_fraction(x))

  def _variable_summaries(var):
    """Attach a lot of summaries to a Tensor."""
    if not tf.get_variable_scope().reuse:
      name = var.op.name
      with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar(name + '/mean', mean)
        stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
        tf.summary.scalar(name + '/sttdev', stddev)
        l2norm = tf.sqrt(tf.reduce_sum(tf.square(var)))
        tf.summary.scalar(name + '/l2norm', l2norm)
        tf.summary.histogram(name, var)



  # def _variable_summary(self,var):
  #   """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  #   varriable_name=var.op.name
  #   mean = tf.reduce_mean(var)
  #   tf.summary.scalar(varriable_name+'/mean', mean)
  #   stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
  #   tf.summary.scalar(varriable_name+'/stddev', stddev)
  #   l2norm = tf.sqrt(tf.reduce_sum(tf.square(var)))
  #   tf.summary.scalar(varriable_name + '/l2norm', l2norm)
  #   tf.summary.histogram(varriable_name+'/histogram', var)

  def _variable_on_cpu(self,name, shape, initializer, pretrain = False, trainable = True):
    """Helper to create a Variable stored on CPU memory.

    Args:
      name: name of the variable
      shape: list of ints
      initializer: initializer for Variable

    Returns:
      Variable Tensor
    """
    with tf.device('/cpu:0'):
      var = tf.get_variable(name, shape, initializer=initializer)
      #self.var_rename['inference/' + var.op.name] = var #for translate
      # print(var.op.name)
    if  tf.get_variable_scope().reuse == False:
        if pretrain:
          self.pretrain_var_collection.append(var)
        else:
          self.initial_var_collection.append(var)
        if trainable:
          self.trainable_var_collection.append(var)

    return var




  def _variable_with_weight_decay(self,name, shape, wd, pretrain = False, bilinear = False, trainable = True):
    """Helper to create an initialized Variable with weight decay.

    Note that the Variable is initialized with a truncated normal distribution.
    A weight decay is added only if one is specified.

    Args:
      name: name of the variable
      shape: list of ints
      stddev: standard deviation of a truncated Gaussian
      wd: add L2Loss weight decay multiplied by this float. If None, weight
          decay is not added for this Variable.

    Returns:
      Variable Tensor
    """

    if bilinear:
      weights = self.get_bilinear(shape)
      #print(weights[:,:,1,1])
      initializer = tf.constant_initializer(value=weights, dtype=tf.float32)
    else:
      initializer = tf.contrib.layers.xavier_initializer()
    var = self._variable_on_cpu(name, shape, initializer, pretrain, trainable)

    if wd and not tf.get_variable_scope().reuse:
      weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
      #weight_decay = tf.reduce_mean((var**2)*wd, name='weight_loss')
      weight_decay.set_shape([])
      tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, weight_decay)
    return var

  def conv_layer(self,scope_name,inputs, kernel_size,num_features, stride=1, linear = False, pretrain = False, batchnormalization = False, trainable = True):
    """convolutional layer
    Args:
    input: 4 - D
    tensor[batch_size, height, width, depth]
    scope: variable_scope
    name
    kernel_size: [k_height, k_width]
    stride: int32

  Return:
  output: 4 - D
  tensor[batch_size, height / stride, width / stride, out_channels]
  """
    with tf.variable_scope(scope_name) as scope:
      input_channels = inputs.get_shape()[3].value
      weights = self._variable_with_weight_decay('weights', shape=[kernel_size,kernel_size,input_channels,num_features], wd=self.weight_decay, pretrain = pretrain, trainable = trainable)
      biases = self._variable_on_cpu('biases',[num_features],tf.constant_initializer(self.weight_init), pretrain, trainable)
      pad_size = kernel_size // 2
      pad_mat = np.array([[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]])
      inputs_pad = tf.pad(inputs, pad_mat)
      conv = tf.nn.conv2d(inputs_pad, weights, strides=[1, stride, stride, 1], padding='VALID')
      self.testvar = biases
      conv_biased = tf.nn.bias_add(conv, biases,  name='linearout')
      if batchnormalization:
        conv_biased = tf.layers.batch_normalization(conv_biased, training = self.is_training)
      if linear:
        return conv_biased
      conv_rect = self.leaky_relu(conv_biased,self.leaky_alpha )
      # scope.reuse_variables()
      return conv_rect

  def transpose_conv_layer(self,scope_name,inputs, kernel_size,num_features, stride, linear = False, pretrain = False, trainable = True):
    #Filter size:A 4-D Tensor with the same type as value and shape [height, width, output_channels, in_channels],different from conv.
    with tf.variable_scope(scope_name) as scope:
      input_channels = inputs.get_shape()[3].value
      weights = self._variable_with_weight_decay('weights', shape=[kernel_size,kernel_size,num_features,input_channels], wd=self.weight_decay, pretrain = pretrain, bilinear = False, trainable = trainable)
      biases = self._variable_on_cpu('biases',[num_features],tf.constant_initializer(self.weight_init), pretrain, trainable)
      # scope.reuse_variables()
      batch_size = tf.shape(inputs)[0]
      output_shape = tf.stack([tf.shape(inputs)[0], tf.shape(inputs)[1]*stride, tf.shape(inputs)[2]*stride, num_features])
      conv = tf.nn.conv2d_transpose(inputs, weights, output_shape, strides=[1,stride,stride,1], padding='SAME')
      conv_biased = tf.nn.bias_add(conv, biases, name='linearout')
      if linear:
        return conv_biased
      conv_rect = self.leaky_relu(conv_biased,self.leaky_alpha )
      return conv_rect

  def max_pool(self,scope_name, input, kernel_size, stride):
    """max_pool layer
    Args:
      input: 4-D tensor [batch_zie, height, width, depth]
      kernel_size: [k_height, k_width]
      stride: int32
    Return:
      output: 4-D tensor [batch_size, height/stride, width/stride, depth]
     """
    with tf.variable_scope(scope_name) as scope:
      pool = tf.nn.max_pool(input, ksize=[1, kernel_size, kernel_size, 1], strides=[1, stride, stride, 1], padding='SAME',name='pooling')
    return pool


  def fc_layer(self,scope_name,inputs, hiddens, flat = False, linear = False, pretrain = False, trainable = True):
    with tf.variable_scope(scope_name) as scope:
      input_shape = inputs.get_shape().as_list()
      if flat:
        dim = input_shape[1]*input_shape[2]*input_shape[3]
        inputs_processed = tf.reshape(inputs, [-1,dim])
      else:
        dim = input_shape[1]
        inputs_processed = inputs

      weights = self._variable_with_weight_decay('weights', shape=[dim,hiddens], wd=self.weight_decay, pretrain=pretrain, trainable = trainable)
      biases = self._variable_on_cpu('biases', [hiddens], tf.constant_initializer(self.weight_init), pretrain, trainable)
      # scope.reuse_variables()
      ip = tf.add(tf.matmul(inputs_processed, weights), biases, name='linearout')
      if linear:
       return ip
      fc_relu =  self.leaky_relu(ip,self.leaky_alpha )
      return fc_relu

  def leaky_conv(self, net_in, n_filter, filter_size, strides, name, pretrain=True, trainable=True):
    return self.conv_layer(scope_name=name, inputs=net_in, kernel_size=filter_size, num_features=n_filter,
                               stride=strides, linear=False, pretrain=pretrain,
                               batchnormalization=False, trainable=trainable)

  def leaky_deconv(self, name, input_layer, n_filter, out_size):
    return self.transpose_conv_layer(scope_name=name, inputs=input_layer, kernel_size=4, num_features=n_filter,
                                         stride=2, linear=False, pretrain=True, trainable=True)

  def upsample(self, name, input_layer, out_size):
    return self.transpose_conv_layer(scope_name=name, inputs=input_layer, kernel_size=4, num_features=2,
                                         stride=2, linear=True, pretrain=True, trainable=True)

  def flow(self, name, input_layer, filter_size=3):
        return self.conv_layer(scope_name=name, inputs=input_layer, kernel_size=filter_size, num_features=2,
                               stride=1, linear=True, pretrain=True, batchnormalization=False, trainable=True)

  def conv_mask(self, net_in, mask):
      tempsize = net_in.get_shape().as_list()
      net_in_mask = tf.image.resize_images(mask, [tempsize[1], tempsize[2]])
      #print(net_in_mask.get_shape().as_list())
      return net_in * net_in_mask


# BasicConvLSTMCell

In [5]:


class ConvRNNCell(object):
    """Abstract object representing an Convolutional RNN cell.
    """

    def __call__(self, inputs, state, scope=None):
        """Run this RNN cell on inputs, starting from the given state.
        """
        raise NotImplementedError("Abstract method")

    @property
    def state_size(self):
        """size(s) of state(s) used by this cell.
        """
        raise NotImplementedError("Abstract method")

    @property
    def output_size(self):
        """Integer or TensorShape: size of outputs produced by this cell."""
        raise NotImplementedError("Abstract method")

    def zero_state(self, batch_size, hiddennum, dtype):
        """Return zero-filled state tensor(s).
        Args:
          batch_size: int, float, or unit Tensor representing the batch size.
          dtype: the data type to use for the state.
        Returns:
          tensor of shape '[batch_size x shape[0] x shape[1] x num_features]
          filled with zeros
        """

        shape = self.shape
        num_features = self.num_features
        zeros = tf.zeros([batch_size, shape[0], shape[1], num_features * hiddennum])
        return zeros


class BasicConvGRUCell(ConvRNNCell):
    """Basic Conv LSTM recurrent network cell. The
    """

    def __init__(self, shape, filter_size, num_features, forget_bias=1.0, input_size=None,
                 state_is_tuple=False, activation=tf.nn.tanh):
        """Initialize the basic Conv LSTM cell.
        Args:
          shape: int tuple thats the height and width of the cell
          filter_size: int tuple thats the height and width of the filter
          num_features: int thats the depth of the cell
          forget_bias: float, The bias added to forget gates (see above).
          input_size: Deprecated and unused.
          state_is_tuple: If True, accepted and returned states are 2-tuples of
            the `c_state` and `m_state`.  If False, they are concatenated
            along the column axis.  The latter behavior will soon be deprecated.
          activation: Activation function of the inner states.
        """
        # if not state_is_tuple:
        # logging.warn("%s: Using a concatenated state is slower and will soon be "
        #             "deprecated.  Use state_is_tuple=True.", self)
        if input_size is not None:
            tf.logging.warn("%s: The input_size parameter is deprecated.", self)
        self.shape = shape
        self.filter_size = filter_size
        self.num_features = num_features
        self._forget_bias = forget_bias
        self._state_is_tuple = state_is_tuple
        self._activation = activation
        self.trainable_var_collection = []

    @property
    def state_size(self):
        return (tf.contrib.rnn.LSTMStateTuple(self._num_units, self._num_units)
                if self._state_is_tuple else 2 * self._num_units)

    @property
    def output_size(self):
        return self._num_units

    def __call__(self, inputs, h, mask_in, mask_h, dp_in, dp_h, scope=None):
        """Long short-term memory cell (LSTM)."""
        # Parameters of gates are concatenated into one multiply for efficiency.
        assert mask_in.get_shape().as_list()[:-1] == inputs.get_shape().as_list() and mask_h.get_shape().as_list()[
                                                                                      :-1] == inputs.get_shape().as_list()
        # print(mask_in.get_shape().as_list()[-1])
        assert mask_in.get_shape().as_list()[-1] == 3 and mask_h.get_shape().as_list()[-1] == 3



            # concat = self._conv_linear([inputs, h], self.filter_size, self.num_features * 4, True,scope = scope)

            # i = input_gate, j = new_input, f = forget_gate, o = output_gate
            # i, j, f, o = tf.split(axis=3, num_or_size_splits=4, value=concat)

        z_h = self._conv_linear([h * mask_h[..., 0]], self.filter_size, self.num_features, False,
                                scope=scope + 'h1') * (1 / dp_h)
        r_h = self._conv_linear([h * mask_h[..., 1]], self.filter_size, self.num_features, False,
                                scope=scope + 'h2') * (1 / dp_h)
        th_h = self._conv_linear([h * mask_h[..., 2]], self.filter_size, self.num_features, False,
                                scope=scope + 'h3') * (1 / dp_h)


        z_in = self._conv_linear([inputs * mask_in[..., 0]], self.filter_size, self.num_features, True,
                                 scope=scope + 'i1') * (1 / dp_in)
        r_in = self._conv_linear([inputs * mask_in[..., 1]], self.filter_size, self.num_features, True,
                                 scope=scope + 'i2') * (1 / dp_in)
        th_in = self._conv_linear([inputs * mask_in[..., 2]], self.filter_size, self.num_features, True,
                                 scope=scope + 'i3') * (1 / dp_in)

        z_t = tf.nn.sigmoid(z_h + z_in)
        r_t = tf.nn.sigmoid(r_h + r_in)
        th_t = self._activation(r_t * th_h + th_in)
        new_h = (1 - z_t) * h + z_t * th_t
        return new_h

    def _conv_linear(self, args, filter_size, num_features, bias, bias_start=0.0, scope=None):
        """convolution:
        Args:
          args: a 4D Tensor or a list of 4D, batch x n, Tensors.
          filter_size: int tuple of filter height and width.
          num_features: int, number of features.
          bias_start: starting value to initialize the bias; 0 by default.
          scope: VariableScope for the created subgraph; defaults to "Linear".
        Returns:
          A 4D Tensor with shape [batch h w num_features]
        Raises:
          ValueError: if some of the arguments has unspecified or wrong shape.
        """

        # Calculate the total size of arguments on dimension 1.
        total_arg_size_depth = 0
        shapes = [a.get_shape().as_list() for a in args]
        for shape in shapes:
            if len(shape) != 4:
                raise ValueError("Linear is expecting 4D arguments: %s" % str(shapes))
            if not shape[3]:
                raise ValueError("Linear expects shape[4] of arguments: %s" % str(shapes))
            else:
                total_arg_size_depth += shape[3]

        dtype = [a.dtype for a in args][0]

        # Now the computation.
        with tf.variable_scope(scope):
            matrix = tf.get_variable(
                "weights", [filter_size[0], filter_size[1], total_arg_size_depth, num_features], dtype=dtype)
            if len(args) == 1:
                res = tf.nn.conv2d(args[0], matrix, strides=[1, 1, 1, 1], padding='SAME')
            else:
                res = tf.nn.conv2d(tf.concat(axis=3, values=args), matrix, strides=[1, 1, 1, 1], padding='SAME')
            if not bias:
                return res
            bias_term = tf.get_variable(
                "biases", [num_features],
                dtype=dtype,
                initializer=tf.constant_initializer(
                    bias_start, dtype=dtype))
            if tf.get_variable_scope().reuse == False:
                self.trainable_var_collection.append(matrix)
                self.trainable_var_collection.append(bias_term)

        return res + bias_term





class BasicConvLSTMCell(ConvRNNCell):
    """Basic Conv LSTM recurrent network cell. The
    """

    def __init__(self, shape, filter_size, num_features, forget_bias=1.0, input_size=None,
                 state_is_tuple=False, activation=tf.nn.tanh):
        """Initialize the basic Conv LSTM cell.
        Args:
          shape: int tuple thats the height and width of the cell
          filter_size: int tuple thats the height and width of the filter
          num_features: int thats the depth of the cell
          forget_bias: float, The bias added to forget gates (see above).
          input_size: Deprecated and unused.
          state_is_tuple: If True, accepted and returned states are 2-tuples of
            the `c_state` and `m_state`.  If False, they are concatenated
            along the column axis.  The latter behavior will soon be deprecated.
          activation: Activation function of the inner states.
        """
        # if not state_is_tuple:
        # logging.warn("%s: Using a concatenated state is slower and will soon be "
        #             "deprecated.  Use state_is_tuple=True.", self)
        if input_size is not None:
            tf.logging.warn("%s: The input_size parameter is deprecated.", self)
        self.shape = shape
        self.filter_size = filter_size
        self.num_features = num_features
        self._forget_bias = forget_bias
        self._state_is_tuple = state_is_tuple
        self._activation = activation
        self.trainable_var_collection = []

    @property
    def state_size(self):
        return (tf.contrib.rnn.LSTMStateTuple(self._num_units, self._num_units)
                if self._state_is_tuple else 2 * self._num_units)

    @property
    def output_size(self):
        return self._num_units

    def __call__(self, inputs, state, mask_in, mask_h, dp_in, dp_h, scope=None):
        """Long short-term memory cell (LSTM)."""
        # Parameters of gates are concatenated into one multiply for efficiency.
        assert mask_in.get_shape().as_list()[:-1] == inputs.get_shape().as_list() and mask_h.get_shape().as_list()[
                                                                                      :-1] == inputs.get_shape().as_list()
        # print(mask_in.get_shape().as_list()[-1])
        assert mask_in.get_shape().as_list()[-1] == 4 and mask_h.get_shape().as_list()[-1] == 4
        if self._state_is_tuple:
            c, h = state
        else:
            c, h = tf.split(axis=3, num_or_size_splits=2, value=state)

            # concat = self._conv_linear([inputs, h], self.filter_size, self.num_features * 4, True,scope = scope)

            # i = input_gate, j = new_input, f = forget_gate, o = output_gate
            # i, j, f, o = tf.split(axis=3, num_or_size_splits=4, value=concat)

        i_h = self._conv_linear([h * mask_h[..., 0]], self.filter_size, self.num_features, False,
                                scope=scope + 'h1') * (1 / dp_h)
        j_h = self._conv_linear([h * mask_h[..., 1]], self.filter_size, self.num_features, False,
                                scope=scope + 'h2') * (1 / dp_h)
        f_h = self._conv_linear([h * mask_h[..., 2]], self.filter_size, self.num_features, False,
                                scope=scope + 'h3') * (1 / dp_h)
        o_h = self._conv_linear([h * mask_h[..., 3]], self.filter_size, self.num_features, False,
                                scope=scope + 'h4') * (1 / dp_h)
        i_in = self._conv_linear([inputs * mask_in[..., 0]], self.filter_size, self.num_features, True,
                                 scope=scope + 'i1') * (1 / dp_in)
        j_in = self._conv_linear([inputs * mask_in[..., 1]], self.filter_size, self.num_features, True,
                                 scope=scope + 'i2') * (1 / dp_in)
        f_in = self._conv_linear([inputs * mask_in[..., 2]], self.filter_size, self.num_features, True,
                                 scope=scope + 'i3') * (1 / dp_in)
        o_in = self._conv_linear([inputs * mask_in[..., 3]], self.filter_size, self.num_features, True,
                                 scope=scope + 'i4') * (1 / dp_in)

        new_c = (c * tf.nn.sigmoid(f_h + f_in + self._forget_bias) + tf.nn.sigmoid(i_h + i_in) *
                 self._activation(j_h + j_in))
        new_h = self._activation(new_c) * tf.nn.sigmoid(o_h + o_in)

        if self._state_is_tuple:
            new_state = tf.contrib.rnn.LSTMStateTuple(new_c, new_h)
        else:
            new_state = tf.concat(axis=3, values=[new_c, new_h])
        return new_h, new_state

    def _conv_linear(self, args, filter_size, num_features, bias, bias_start=0.0, scope=None):
        """convolution:
        Args:
          args: a 4D Tensor or a list of 4D, batch x n, Tensors.
          filter_size: int tuple of filter height and width.
          num_features: int, number of features.
          bias_start: starting value to initialize the bias; 0 by default.
          scope: VariableScope for the created subgraph; defaults to "Linear".
        Returns:
          A 4D Tensor with shape [batch h w num_features]
        Raises:
          ValueError: if some of the arguments has unspecified or wrong shape.
        """

        # Calculate the total size of arguments on dimension 1.
        total_arg_size_depth = 0
        shapes = [a.get_shape().as_list() for a in args]
        for shape in shapes:
            if len(shape) != 4:
                raise ValueError("Linear is expecting 4D arguments: %s" % str(shapes))
            if not shape[3]:
                raise ValueError("Linear expects shape[4] of arguments: %s" % str(shapes))
            else:
                total_arg_size_depth += shape[3]

        dtype = [a.dtype for a in args][0]

        # Now the computation.
        with tf.variable_scope(scope):
            matrix = tf.get_variable(
                "weights", [filter_size[0], filter_size[1], total_arg_size_depth, num_features], dtype=dtype)
            if len(args) == 1:
                res = tf.nn.conv2d(args[0], matrix, strides=[1, 1, 1, 1], padding='SAME')
            else:
                res = tf.nn.conv2d(tf.concat(axis=3, values=args), matrix, strides=[1, 1, 1, 1], padding='SAME')
            if not bias:
                return res
            bias_term = tf.get_variable(
                "biases", [num_features],
                dtype=dtype,
                initializer=tf.constant_initializer(
                    bias_start, dtype=dtype))
            if tf.get_variable_scope().reuse == False:
                self.trainable_var_collection.append(matrix)
                self.trainable_var_collection.append(bias_term)

        return res + bias_term


# Model

In [6]:

class Net(BasicNet):
  # image_size = 448
  batch_size = 16
  framenum = 16
  maximgbatch = 4
  init_learning_rate = 10**(-4)
  eps = 1e-7
  gapnum = 5
  salmask_lb = 0.5 #mask cam be salmask_lb~1
  dp_in = 0.25
  dp_h = 0.25
  # num_classes = 20
  cell_size = 7
  # boxes_per_cell = 2
  def __init__(self):
    super(Net, self).__init__() # init the fatther class of YoloTinyNet
    self.global_step = tf.Variable(0, trainable=False)
    self.initial_var_collection.append(self.global_step )
    self.out = []
    self.predict = []
    self.loss = []
    self.loss_gt = []
    self.re = []
    self.loss_gt2 = []
    self.yolofeatures_colllection = []
    self.flowfeatures_colllection = []
    self.startflagcnn = True
    #process params

  def YOLO_tiny_inference(self, images):  # pre128
      cnnpretrain = True
      cnntrainable = False
      self.batch_size = images.get_shape()[0].value
      conv_1 = self.conv_layer('conv1', images, 3, 16, stride=1, pretrain=cnnpretrain, batchnormalization=True,
                               trainable=cnntrainable)
      pool_2 = self.max_pool('pool2', conv_1, 2, stride=2)
      conv_3 = self.conv_layer('conv3', pool_2, 3, 32, stride=1, pretrain=cnnpretrain, batchnormalization=True,
                               trainable=cnntrainable)
      pool_4 = self.max_pool('pool4', conv_3, 2, stride=2)
      conv_5 = self.conv_layer('conv5', pool_4, 3, 64, stride=1, pretrain=cnnpretrain, batchnormalization=True,
                               trainable=cnntrainable)
      pool_6 = self.max_pool('pool6', conv_5, 2, stride=2)
      conv_7 = self.conv_layer('conv7', pool_6, 3, 128, stride=1, pretrain=cnnpretrain, batchnormalization=True,
                               trainable=cnntrainable)
      pool_8 = self.max_pool('pool8', conv_7, 2, stride=2)
      conv_9 = self.conv_layer('conv9', pool_8, 3, 256, stride=1, pretrain=cnnpretrain, batchnormalization=True,
                               trainable=cnntrainable)
      pool_10 = self.max_pool('pool10', conv_9, 2, stride=2)
      conv_11 = self.conv_layer('conv11', pool_10, 3, 512, stride=1, pretrain=cnnpretrain, batchnormalization=True,
                                trainable=cnntrainable)
      pool_12 = self.max_pool('pool12', conv_11, 2, stride=2)
      conv_13 = self.conv_layer('conv13', pool_12, 3, 1024, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
      conv_14 = self.conv_layer('conv14', conv_13, 3, 1024, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
      conv_15 = self.conv_layer('conv15', conv_14, 3, 1024, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
      temp_conv = tf.transpose(conv_15, (0, 3, 1, 2))
      fc_16 = self.fc_layer('fc16', temp_conv, 256, flat=True, pretrain=cnnpretrain, trainable=cnntrainable)
      fc_17 = self.fc_layer('fc17', fc_16, 4096, flat=False, pretrain=cnnpretrain, trainable=cnntrainable)
      fc_18 = self.fc_layer('fc18', fc_17, 1470, flat=False, linear=True, pretrain=cnnpretrain, trainable=cnntrainable)

      highFeature = tf.reshape(fc_18, [fc_18.get_shape()[0].value, self.cell_size, self.cell_size, -1])

      conv_15_2 = self.conv_layer('conv_15_2', conv_15, 1, 128, stride=1,pretrain=cnnpretrain, trainable=cnntrainable)
      conv_11_2 = self.conv_layer('conv_11_2', conv_11, 1, 128, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
      conv_9_2 = self.conv_layer('conv_9_2', conv_9, 1, 128, stride=1,pretrain=cnnpretrain, trainable=cnntrainable)
      # conv_7_2 = self.conv_layer('conv_7_2', conv_7, 1, 256, stride=1, pretrain=False)
      tempsize = conv_9.get_shape().as_list()
      newconv_7 = tf.image.resize_images(conv_7, [tempsize[1], tempsize[2]])
      newconv_9 = tf.image.resize_images(conv_9_2, [tempsize[1], tempsize[2]])
      newconv_11_2 = tf.image.resize_images(conv_11_2, [tempsize[1], tempsize[2]])
      newconv_15_2 = tf.image.resize_images(conv_15_2, [tempsize[1], tempsize[2]])
      highFeature = tf.image.resize_images(highFeature, [tempsize[1], tempsize[2]])
      FeatureMap = tf.concat([newconv_7, newconv_9, newconv_11_2, newconv_15_2, highFeature], axis=3)
      weight_mask = tf.constant(self.get_centermask(FeatureMap.get_shape().as_list()), dtype=FeatureMap.dtype)
      FeatureMap = FeatureMap * weight_mask
      return FeatureMap

  def Coarse_salmap(self, Yolofeature):  #  tiny pregen256 fea28_128
    cnnpretrain = True
    cnntrainable = False
    conv_19 = self.conv_layer('conv_19', Yolofeature, 3, 512, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
    conv_20 = self.conv_layer('conv_20', conv_19, 1, 256, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
    conv_21 = self.conv_layer('conv_21', conv_20, 3, 128, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
    conv_22 = self.conv_layer('conv_22', conv_21, 1, 128, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
    deconv_23 = self.transpose_conv_layer('deconv_23', conv_22, 4, 16, stride=2, pretrain=cnnpretrain, trainable=cnntrainable)
    deconv_24 = self.transpose_conv_layer('deconv_24', deconv_23, 4, 1, stride=2, linear=True, pretrain=cnnpretrain, trainable=cnntrainable)

    return deconv_24

  def Final_inference(self, cat1, cat2):
      cnnpretrain = True
      cnntrainable = False
      MyFeature = tf.concat([cat1, cat2], axis=3)
      Lastconv_1 = self.conv_layer('Lastconv_1', MyFeature, 3, 512, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
      Lastconv_2 = self.conv_layer('Lastconv_2', Lastconv_1, 1, 512, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
      Lastconv_3 = self.conv_layer('Lastconv_3', Lastconv_2, 3, 256, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
      Lastconv_4 = self.conv_layer('Lastconv_4', Lastconv_3, 1, 128, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)

      return Lastconv_4

  def flownet_with_conv(self, x1, x2, mask):
        cnnpretrain = True
        cnntrainable = False
        input = tf.concat([x1, x2], axis=3, name='FNinput')
        conv_1 = self.leaky_conv(input, 64, 7, 2, 'FNconv1', pretrain=cnnpretrain, trainable=cnntrainable)
        conv_1 = self.conv_mask(conv_1, mask)
        conv_2 = self.leaky_conv(conv_1, 128, 5, 2, 'FNconv2', pretrain=cnnpretrain, trainable=cnntrainable)
        conv_2 = self.conv_mask(conv_2, mask)
        conv_3 = self.leaky_conv(conv_2, 256, 5, 2, 'FNconv3', pretrain=cnnpretrain, trainable=cnntrainable)
        conv_3 = self.conv_mask(conv_3, mask)
        conv_3_1 = self.leaky_conv(conv_3, 256, 3, 1, 'FNconv3_1', pretrain=cnnpretrain, trainable=cnntrainable)
        conv_3_1 = self.conv_mask(conv_3_1, mask)
        conv_4 = self.leaky_conv(conv_3_1, 512, 3, 2, 'FNconv4', pretrain=cnnpretrain, trainable=cnntrainable)
        conv_4 = self.conv_mask(conv_4, mask)
        conv_4_1 = self.leaky_conv(conv_4, 512, 3, 1, 'FNconv4_1', pretrain=cnnpretrain, trainable=cnntrainable)
        # conv_4_1 = self.conv_mask(conv_4_1, mask)
        conv_5 = self.leaky_conv(conv_4_1, 512, 3, 2, 'FNconv5', pretrain=cnnpretrain, trainable=cnntrainable)
        # conv_5 = self.conv_mask(conv_5, mask)
        conv_5_1 = self.leaky_conv(conv_5, 512, 3, 1, 'FNconv5_1', pretrain=cnnpretrain, trainable=cnntrainable)
        # conv_5_1 = self.conv_mask(conv_5_1, mask)
        conv_6 = self.leaky_conv(conv_5_1, 1024, 3, 2, 'FNconv6', pretrain=cnnpretrain, trainable=cnntrainable)
        # conv_6 = self.conv_mask(conv_6, mask)
        conv_6_1 = self.leaky_conv(conv_6, 1024, 3, 1, 'FNconv6_1', pretrain=cnnpretrain, trainable=cnntrainable)
        # conv_6_1 = self.conv_mask(conv_6_1, mask)
        out_cat_size = conv_4.get_shape().as_list()

        Downconv_6_1 = self.conv_layer('FNDownconv_6_1', conv_6_1, 3, 128, stride=1,pretrain=cnnpretrain, trainable=cnntrainable)
        Downconv_5_1 = self.conv_layer('FNDownconv_5_1', conv_5_1, 3, 128, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
        Downconv_4_1 = self.conv_layer('FNDownconv_4_1', conv_4_1, 3, 128, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
        Downconv_3_1 = self.conv_layer('FNDownconv_3_1', conv_3_1, 3, 128, stride=1, pretrain=cnnpretrain, trainable=cnntrainable)
        conv_6_1_cat = tf.image.resize_images(Downconv_6_1, [out_cat_size[1],out_cat_size[2]])
        conv_5_1_cat = tf.image.resize_images(Downconv_5_1, [out_cat_size[1],out_cat_size[2]])
        conv_4_1_cat = tf.image.resize_images(Downconv_4_1, [out_cat_size[1],out_cat_size[2]])
        conv_3_1_cat = tf.image.resize_images(Downconv_3_1, [out_cat_size[1],out_cat_size[2]])
        concat_out = tf.concat([conv_6_1_cat, conv_5_1_cat, conv_4_1_cat, conv_3_1_cat], axis=3, name='FNconcat_out')

        return concat_out


  def inference(self, videoslides, mask_in, mask_h): #videoslides: [batch framenum h w num_features]
    with tf.variable_scope('inference'):
        shapes = videoslides.get_shape().as_list()
       #shapes2 = GTs.get_shape().as_list()
        assert len(shapes)==5
        self.batch_size =  videoslides.get_shape()[0].value
        #self.framenum = videoslides.get_shape()[1].value
        # assert self.framenum % self.maximgbatch == 0 # frmaenum shoube be the multiple of maximgbatch   scope = 'layer_1'
        with tf.variable_scope('conv_lstm', initializer=tf.random_uniform_initializer(-.01, 0.1)):
            # cell_1 = BasicConvLSTMCell.BasicConvLSTMCell([56, 56], [3, 3], 128, state_is_tuple = False)  # input size,fliter size, input channals
            # cell_2 = BasicConvLSTMCell.BasicConvLSTMCell([56, 56], [3, 3], 128, state_is_tuple = False)  # input size,fliter size, input channals
            cell_1 = BasicConvLSTMCell([28, 28], [3, 3], 128,
                                                         state_is_tuple=False)  # input size,fliter size, input channals
            cell_2 = BasicConvLSTMCell([28, 28], [3, 3], 128,
                                                         state_is_tuple=False)  # input size,fliter size, input channals

            new_state_1 = cell_1.zero_state(self.batch_size, 2, tf.float32)
            new_state_2 = cell_2.zero_state(self.batch_size, 2, tf.float32)
           # print(videoslides.get_shape().as_list())
        for indexframe in range(self.framenum):
            frame = videoslides[:, indexframe, ...]
            #print(indexframe+self.gapnum)
            frame_gap = videoslides[:, indexframe+self.gapnum, ...]
            #GTframe = GTs[:, indexframe, ...]
            Yolo_features = self.YOLO_tiny_inference(frame)
            Presalmap = self.Coarse_salmap(Yolo_features)
            if self.startflagcnn == True:
                self.yolofeatures_colllection = self.pretrain_var_collection
                self.pretrain_var_collection = []
            salmask = self._normlized_0to1(Presalmap)
            salmask = salmask*(1-self.salmask_lb)+self.salmask_lb
            Flow_features = self.flownet_with_conv(frame, frame_gap, salmask)
            CNNout = self.Final_inference(Yolo_features, Flow_features)
            if self.startflagcnn == True:
                self.flowfeatures_colllection = self.pretrain_var_collection
            y_1, new_state_1 = cell_1(CNNout, new_state_1,mask_in[...,0:4], mask_h[...,0:4], self.dp_in, self.dp_h, 'lstm_layer1')
            y_2, new_state_2 = cell_2(y_1, new_state_2,mask_in[...,4:8], mask_h[...,4:8], self.dp_in, self.dp_h, 'lstm_layer2')
            deconv = self.transpose_conv_layer('deconv', y_2, 4, 16, stride=2, pretrain=False, trainable=True)
            deconv2 = self.transpose_conv_layer('deconv2', deconv, 4, 1, stride=2, linear=True, pretrain=False, trainable=True)
            if self.startflagcnn == True:
                tf.get_variable_scope().reuse_variables()
                self.trainable_var_collection.extend(cell_1.trainable_var_collection)
                self.trainable_var_collection.extend(cell_2.trainable_var_collection)
                self.startflagcnn = False

            output = self._normlized_0to1(deconv2)
            #norm_GT = self._normlized(GTframe)
            norm_output = self._normlized(output)
           # frame_loss = norm_GT * tf.log(self.eps + norm_GT / (norm_output + self.eps))
            #frame_loss = tf.reduce_sum(frame_loss) / norm_GT.get_shape()[0].value
           # tf.add_to_collection('losses', frame_loss)
            output = tf.expand_dims(output, 1)
            if indexframe == 0:
                tempout = output
            else:
                tempout = tf.concat([tempout, output], axis=1)
        self.out = tempout




  def _normlized(self, mat): # tensor [batch_size, image_height, image_width, channels] normalize each fea map
    mat_shape = mat.get_shape().as_list()
    tempsum = tf.reduce_sum(mat, axis=1)
    tempsum = tf.reduce_sum(tempsum, axis=1) + self.eps
    tempsum = tf.reshape(tempsum, [-1, 1, 1, mat_shape[3]])
    return mat / tempsum

  def _normlized_0to1(self, mat): # tensor [batch_size, image_height, image_width, channels] normalize each fea map
    mat_shape = mat.get_shape().as_list()
    tempmin = tf.reduce_min(mat, axis=1)
    tempmin= tf.reduce_min(tempmin, axis=1)
    tempmin = tf.reshape(tempmin, [-1, 1, 1, mat_shape[3]])
    tempmat = mat - tempmin
    tempmax = tf.reduce_max(tempmat, axis=1)
    tempmax = tf.reduce_max(tempmax, axis=1) + self.eps
    tempmax = tf.reshape(tempmax, [-1, 1, 1, mat_shape[3]])
    return tempmat / tempmax

  def _loss(self):
    weight_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES, scope=None)
    loss_weight = tf.add_n(weight_loss)
    loss_kl = tf.get_collection('losses', scope=None)
    loss_kl = tf.add_n(loss_kl)/self.framenum
    # self.out = self.predict
    tf.summary.scalar('loss_weight', loss_weight)
    tf.summary.scalar('loss_kl', loss_kl)
    self.loss_gt = loss_kl
    self.loss = loss_kl + loss_weight



  def _train(self):
    # learning_rate = tf.train.exponential_decay(self.init_learning_rate, self.global_step,
    #                                            100000, 0.95, staircase=True)
    # with tf.variable_scope('trainer'):
        opt = tf.train.AdamOptimizer(self.init_learning_rate,beta1=0.9, beta2=0.999, epsilon=1e-08)
        # for var in self.trainable_var_collection:
        #   print(var.op.name)
        grads = opt.compute_gradients(self.loss,var_list = self.trainable_var_collection)
        apply_gradient_op = opt.apply_gradients(grads, global_step=self.global_step)
        #apply_gradient_op = tf.train.AdamOptimizer(self.init_learning_rate).minimize(self.loss)
        self.train = apply_gradient_op
        return apply_gradient_op


# Test

In [ ]:
cd /content/drive/My Drive/model/

/content/drive/My Drive/model/pretrain


In [ ]:
ls

LSTMconv_prefinal_loss05_dp075_075MC100-200000.index
LSTMconv_prefinal_loss05_dp075_075MC100-200000.meta


In [9]:
batch_size = 1
framesnum = 16
inputDim = 448
input_size = (inputDim, inputDim)
resize_shape = input_size
outputDim = 112
output_size = (outputDim, outputDim)
epoch_num = 15
overlapframe = 5 #0~framesnum+frame_skip

random.seed(a=730)
tf.set_random_seed(730)
frame_skip = 5
dp_in = 1
dp_h = 1

targetname = 'LSTMconv_prefinal_loss05_dp075_075MC100-200000'
VideoName = '/content/drive/My Drive/model/test/cont_5.mp4'
CheckpointFile = '/content/drive/My Drive/model/pretrain/'+ targetname

def _BatchExtraction(VideoCap, batchsize=batch_size, last_input=None, video_start = True):
	if video_start:
		_, frame = VideoCap.read()
		frame = cv2.resize(frame, resize_shape)
		frame = frame.astype(np.float32)
		frame = frame / 255.0 * 2 - 1
		frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
		Input_Batch = frame[np.newaxis, ...]
		for i in range(batchsize - 1):
			_, frame = VideoCap.read()
			frame = cv2.resize(frame, resize_shape)
			frame = frame.astype(np.float32)
			frame = frame / 255.0 * 2 - 1
			frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)     
			frame = frame[np.newaxis, ...]       
			Input_Batch = np.concatenate((Input_Batch, frame), axis=0)
		Input_Batch = Input_Batch[np.newaxis, ...]       
	else:
		Input_Batch = last_input[:,-overlapframe:,...]      
		for i in range(batchsize-overlapframe):
			_, frame = VideoCap.read()   
			frame = cv2.resize(frame, resize_shape)
			frame = frame.astype(np.float32)
			frame = frame / 255.0 * 2 - 1
			frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)          
			frame = frame[np.newaxis,np.newaxis, ...]         
			Input_Batch = np.concatenate((Input_Batch, frame), axis=1)
	return Input_Batch




tf.reset_default_graph()
net = Net()
net.is_training = False
input = tf.placeholder(tf.float32, (batch_size, framesnum + frame_skip, input_size[0], input_size[1], 3))
RNNmask_in = tf.placeholder(tf.float32, (batch_size, 28, 28, 128, 4 * 2))
RNNmask_h = tf.placeholder(tf.float32, (batch_size, 28, 28, 128, 4 * 2))
net.inference(input, RNNmask_in, RNNmask_h)
net.dp_in = dp_in
net.dp_h = dp_h
predicts = net.out

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# sess = tf.Session(config=config)
saver = tf.train.Saver()
saver.restore(sess, CheckpointFile)
VideoName_short = VideoName[:-4]
VideoCap = cv2.VideoCapture(VideoName)
  
VideoSize = (int(VideoCap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(VideoCap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
VideoFrame = int(VideoCap.get(cv2.CAP_PROP_FRAME_COUNT))
print('New video: %s with %d frames and size of (%d, %d)' % (VideoName_short, VideoFrame, VideoSize[1], VideoSize[0]))
fps = float(VideoCap.get(cv2.CAP_PROP_FPS))
videoWriter = cv2.VideoWriter(VideoName_short + '_out.avi', cv2.VideoWriter_fourcc('D', 'I', 'V', 'X'),fps,	output_size, isColor=False)
#cv2.VideoWriter_fourcc('D', 'I', 'V', 'X'),
start_time = time.time()
videostart = True
while VideoCap.get(cv2.CAP_PROP_POS_FRAMES) < VideoFrame - framesnum - frame_skip + overlapframe:
  if videostart:
    Input_Batch = _BatchExtraction(VideoCap, framesnum + frame_skip, video_start=videostart)
    videostart = False
    Input_last = Input_Batch
  else:
    Input_Batch = _BatchExtraction(VideoCap, framesnum + frame_skip, last_input=Input_last,video_start=videostart)
    Input_last = Input_Batch

  mask_in = np.ones((1, 28, 28, 128, 4 * 2))
  mask_h = np.ones((1, 28, 28, 128, 4 * 2))
  np_predict = sess.run(predicts, feed_dict={input: Input_Batch, RNNmask_in: mask_in, RNNmask_h: mask_h})
  for index in range(framesnum):
    Out_frame = np_predict[0,index, :, :, 0]
    Out_frame = Out_frame * 255
    Out_frame = np.uint8(Out_frame)
    videoWriter.write(Out_frame)
    #print("write down")
# restFrame = VideoFrame - VideoCap.get(cv2.CAP_PROP_POS_FRAMES)
#overlap = framesnum - restFrame + overlapframe
#Input_Batch = _BatchExtraction(VideoCap, framesnum + frame_skip, last_input=Input_last,video_start=False)
#mask_in = np.ones((1, 28, 28, 128, 4 * 2))
#mask_h = np.ones((1, 28, 28, 128, 4 * 2))
#np_predict = sess.run(predicts, feed_dict={input: Input_Batch, RNNmask_in: mask_in, RNNmask_h: mask_h})
#for index in range(restFrame):
#	Out_frame = np_predict[0, framesnum - restFrame + index, :, :, 0]
#	Out_frame = Out_frame * 255
#	Out_frame = np.uint8(Out_frame)
#	videoWriter.write(Out_frame)
    
duration = float(time.time() - start_time)
print('Total time for this video %f' % (duration))
  # print(duration)
VideoCap.release()



INFO:tensorflow:Restoring parameters from /content/drive/My Drive/model/pretrain/LSTMconv_prefinal_loss05_dp075_075MC100-200000
New video: /content/drive/My Drive/model/test/cont_5 with 1768 frames and size of (1080, 1920)
Total time for this video 166.621735
